==TODO==

- [ ] 模板编译，两个控件关联同一个变量如何处理
- [ ] vue.top_p时做到按需刷新界面，当前为重新绘制, [constraint-propagate](https://github.com/leilux/SICP-exercises/blob/master/book/p216-constraint-propagate(python%20version).py#L124)
  - model和view连接到constraint
  - （ui操作）控件值变化，observe通过constraint更新其他控件、vm.xxx。其他控件值更新也会触发observe，通过判断当前  constraint是否在notify状态决定是否要调用notify。
  - vm.xxx改变，set()通过constraint更新其他控件，其他控件值更新也会触发observe，通过判断当前constraint是否在notify状态决定是否要调用notify。
  - （ui操作）on_click、methods中函数等导致列表元素个数发生改变，on_click_handler中重新render界面

In [1]:
import ast
import enum
import functools
import re

import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display

In [2]:
def log_debug(msg):
    pass
    # print(msg)

In [64]:
import markdown
css_style = !pygmentize -S default -f html -a .codehilite

class MarkdownViewer(widgets.HTML):
    def __init__(self, value):
        extra = [
            'markdown.extensions.extra',
            'markdown.extensions.codehilite',
            'markdown.extensions.tables',
            # 'markdown.extensions.nl2br',
        ]
        html = markdown.markdown(value, extensions=extra)
        super().__init__(f"<style>{''.join(css_style)}</style>" + html)
        # super().__init__(html)
md = (MarkdownViewer(value='''
## h2
```python
def f():
  pass
```
'''))
display(md)

MarkdownViewer(value='<style>pre { line-height: 125%; }td.linenos .normal { color: inherit; background-color: …

In [65]:
md.value

'<style>pre { line-height: 125%; }td.linenos .normal { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }span.linenos { color: inherit; background-color: transparent; padding-left: 5px; padding-right: 5px; }td.linenos .special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }span.linenos.special { color: #000000; background-color: #ffffc0; padding-left: 5px; padding-right: 5px; }.codehilite .hll { background-color: #ffffcc }.codehilite { background: #f8f8f8; }.codehilite .c { color: #3D7B7B; font-style: italic } /* Comment */.codehilite .err { border: 1px solid #FF0000 } /* Error */.codehilite .k { color: #008000; font-weight: bold } /* Keyword */.codehilite .o { color: #666666 } /* Operator */.codehilite .ch { color: #3D7B7B; font-style: italic } /* Comment.Hashbang */.codehilite .cm { color: #3D7B7B; font-style: italic } /* Comment.Multiline */.codehilite .cp { color: #9C6500 } /* Comment.Preproc */.codehilite .

# VueTemplate

In [15]:
from html.parser import HTMLParser

class Tag:
    AppLayout = "AppLayout".lower()
    Box = "Box".lower()
    HBox = "HBox".lower()
    FloatSlider = "FloatSlider".lower()
    Dropdown = "Dropdown".lower()
    Textarea = "Textarea".lower()
    Button = 'Button'.lower()
    MarkdownViewer = 'MarkdownViewer'.lower()
    Template = 'template'

    container_tags = (AppLayout, Box, HBox, Template)
    leaf_tags = (FloatSlider, Dropdown, Textarea, Button, MarkdownViewer)

    @classmethod
    def is_container(cls, t):
        return t in [i.lower() for i in cls.container_tags]

    @classmethod
    def is_leaf(cls, t):
        return t in [i.lower() for i in cls.leaf_tags]

    @classmethod
    def get_widget(cls, t):
        return getattr(widgets, getattr(cls, t))


class VForStatement:
    def __init__(self, target, iters, index=None):
        self.i = index
        self.target = target
        self.iter = iters

    @classmethod
    def parse(cls, s):
        i_target, iters = [i.strip() for i in s.split('in')]
        i_target = [i.strip() for i in i_target.strip('()').split(',')]
        if len(i_target) == 1:
            i, target = None, i_target[0]
        else:
            i, target = i_target[0], i_target[1]

        return cls(target.strip(), iters, i)

class ForScope:
    def __init__(self, i_val, for_stmt: VForStatement, vm: 'Vue'):
        self.for_stmt = for_stmt
        self.i = i_val
        self.iter = get_attr(vm, for_stmt.iter)
        log_debug(f"get vm.{for_stmt.iter} {self.iter}")
        self.target = self.iter[self.i]


class Directive:
    v_for = 'v-for'
    v_slot = 'v-slot:'
    v_js_link = 'v-js-link'
    v_model = 'v-model'
    event_prefix = '@'
    single_bound_prefix = ':'

    @classmethod
    def is_v_for(cls, attr, value):
        if attr == cls.v_for:
            pass

    @classmethod
    def is_v_slot(cls, attr, value):
        if not attr.startswith(cls.v_slot):
            return False, None
        return True, attr.split(':')[1]

    @classmethod
    def is_v_js_link(cls, attr, value):
        if attr == cls.v_js_link:
            return True, None
        return False, None

    @classmethod
    def is_v_model(cls, attr, value):
        if attr == cls.v_model:
            return True, value
        return False, None

    @classmethod
    def is_event(cls, attr: str, func):
        if not attr.startswith(cls.event_prefix):
            return False, attr, None, None
        event = attr.lstrip(cls.event_prefix)
        match = re.match('(.*)\((.*)\)', func)
        if not match:
            return True, event, func, None

        func = match[1]
        args = [i.strip() for i in match[2].split(',')]
        return True, event, func, args


    @classmethod
    def is_single_bound(cls, attr: str, value):
        if attr.startswith(cls.single_bound_prefix):
            return True, attr.lstrip(cls.single_bound_prefix)
        return False, None


class VueTemplate(HTMLParser):
    """
    https://docs.python.org/zh-cn/3.10/library/ast.html#function-and-class-definitions
    """
    def __init__(self, vm: 'Vue'):
        super().__init__()
        self.vm = vm
        self.widgets = []
        self.widgets_by_id = {}
        self.parent_node_stack = []
        self.v_for_stack = []

    def _get_element_by_id(self, el_id):
        return self.widgets_by_id.get(el_id)

    @property
    def is_in_for_stmt(self):
        return bool(self.v_for_stack)

    def _process_directive(self):
        pass

    def _gen_ast_node(self, tag, attrs, for_scope=None):
        if Tag.is_container(tag):
            node = self._container_tag_enter(tag, attrs, for_scope)
        elif Tag.is_leaf(tag):
            node = self._leaf_tag_enter(tag, attrs, for_scope)
        else:
            # TODO
            # node = self._html_tag_enter(tag, attrs)
            raise Exception(f"tag '{tag}' not support.")
        return node

    def _gen_widget(self, node, for_scope=None):
        if Tag.is_container(node['tag']):
            widget = self._container_tag_exit(node, for_scope)
        else:
            widget = self._leaf_tag_exit(node, for_scope)
        return widget

    def _widget_factory(self, widget_cls, attrs, scopes, v_model_widget='value'):
        kwargs = {}
        on_event = {}
        v_model_vm = None
        v_slot = None
        for attr, value in attrs.items():
            # :attr=value
            ok, var = Directive.is_single_bound(attr, value)
            if ok:
                try:
                    _value = ast.literal_eval(value)
                    kwargs[var] = _value
                    continue
                except:
                    pass
                kwargs[var] = get_attr_from_scopes(scopes, value)
                continue
            # v-model
            ok, model_vm = Directive.is_v_model(attr, value)
            if ok:
                v_model_vm = model_vm
                # print(f'>> get v-model {widget_cls} {v_model_widget} ot {v_model_vm}')
                kwargs[v_model_widget] = get_attr_from_scopes(scopes, v_model_vm)
                continue
            # @click
            ok, event, func, args = Directive.is_event(attr, value)
            if ok:
                on_event[event] = (func, args)
                continue
            # TODO 去除不需要的参数
            ok, slot = Directive.is_v_slot(attr, value)
            if ok:
                v_slot = slot
                continue
            kwargs[attr] = value

        widget = widget_cls(**kwargs)
        if v_slot:
            widget.v_slot = v_slot

        # process directive
        def handle_value_change(_scopes, exp):
            def warp(change):
                ret = set_attr_to_scopes(_scopes, exp, change['new'])
                # print(f'>>> change {exp} to {change["new"]}, {ret}')
            return warp

        if v_model_vm:
            # print(f">> register observe {widget_cls} {v_model_widget} -> {v_model_vm}")
            widget.observe(handle_value_change(scopes, v_model_vm), names=f'{v_model_widget}')

        for _event, (func_name, args_name) in on_event.items():
            func = getattr(self.vm.methods, func_name)
            if args_name:
                args = []
                for arg_name in args_name:
                    if arg_name[0].isalpha() or arg_name[0] == '_':
                        arg = get_attr_from_scopes(scopes, arg_name)
                    else:
                        arg = ast.literal_eval(arg_name)
                    args.append(arg)
                func = functools.partial(func, *args)
            getattr(widget, f'on_{_event}')(func)

        return widget

    def _container_tag_enter(self, tag, attrs, for_scope=None):
        ast_node = {"tag": tag, 'attrs': attrs, 'body': []}
        return ast_node

    def _container_tag_exit(self, node, for_scope=None):
        tag = node['tag']
        attrs = node['attrs']
        widgets_map = {
            Tag.AppLayout: widgets.AppLayout,
            Tag.Box: widgets.Box,
            Tag.HBox: widgets.HBox,
            Tag.Template: widgets.VBox,
        }

        log_debug(f"_container_tag_exit {node}")
        # cur_node = self.parent_node_stack.pop(-1)
        if tag == Tag.AppLayout.lower():
            kwargs = {}
            for child in node['body']:
                log_debug(f"child {child}")
                kwargs[child.v_slot] = child
            widget = widgets_map[tag](**kwargs)
        elif tag == Tag.Box.lower() or tag == Tag.HBox.lower():
            widget = widgets_map[tag](node['body'])
        elif tag == Tag.Template.lower():
            widget = widgets_map[tag](node['body'])
        else:
            raise Exception(f'error: container_tag_exit, {tag} not support.')

        for attr, value in attrs.items():
            ok, slot = Directive.is_v_slot(attr, value)
            if ok:
                widget.v_slot = slot
                continue

        return widget

    def _leaf_tag_enter(self, tag, attrs, for_scope=None):
        ast_node = {"tag": tag, 'attrs': attrs}
        return ast_node

    def _leaf_tag_exit(self, node, for_scope: ForScope = None):
        tag = node['tag']
        scopes = [self.vm]
        if for_scope:
            scopes.append(for_scope)

        widgets_map = {
            Tag.FloatSlider: widgets.FloatSlider,
            Tag.Dropdown: widgets.Dropdown,
            Tag.Button: widgets.Button,
            Tag.Textarea: widgets.Textarea,
            Tag.MarkdownViewer: MarkdownViewer,
        }
        v_model_widget = 'value'
        if tag == Tag.Button:
            v_model_widget = 'description'
        
        return self._widget_factory(widgets_map[tag], node['attrs'], scopes, v_model_widget)


    def _html_tag_enter(self):
        pass

    def _html_tag_exit(self):
        pass

    def _for_stmt_enter(self, for_stmt: VForStatement, tag, attrs, is_body):
        body = []
        for i, target in enumerate(getattr(self.vm, for_stmt.iter)):
            for_scope = (i, target)
            body.append(self._gen_ast_node(tag, attrs, for_scope))

        if is_body:
            ast_node = {
                "tag": 'v_for',
                'v_for_body': for_stmt,
                "body": body,
            }
        else:
            ast_node = {
                "tag": 'v_for',
                'v_for': for_stmt,
                "body": body,
            }
        return ast_node

    def _for_stmt_exit(self, v_for_ast_node, is_body=False):
        _widgets = []
        # v_for_stmt = v_for_ast_node['v_for']
        v_for_stmt = self.v_for_stack[-1]
        _iter = getattr(self.vm, v_for_stmt.iter)
        for i, target in enumerate(_iter):
            # for_scope = (v_for_stmt.i, i, v_for_stmt.target, target, v_for_stmt.target)
            for_scope = ForScope(i, v_for_stmt, self.vm)
            node = v_for_ast_node['body'][i]
            widget = self._gen_widget(node, for_scope)
            _widgets.append(widget)

        return _widgets

    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)
        v_for_stmt = attrs.pop(Directive.v_for, None)
        if v_for_stmt or self.v_for_stack:
            is_header = bool(v_for_stmt)
            if is_header:
                v_for = VForStatement.parse(v_for_stmt)
                self.v_for_stack.append(v_for)
            v_for = self.v_for_stack[-1]
            node = self._for_stmt_enter(v_for, tag, attrs, not is_header)
        else:
            node = self._gen_ast_node(tag, attrs)
        self.parent_node_stack.append(node)

    def handle_endtag(self, tag):
        node = self.parent_node_stack.pop()
        if self.is_in_for_stmt:
            is_body = 'v_for' not in node
            _widgets = self._for_stmt_exit(node, is_body)
            if is_body:
                for i, widget in enumerate(_widgets):
                    self.parent_node_stack[-1]['body'][i]['body'].append(widget)
            else:
                self.parent_node_stack[-1]['body'].extend(_widgets)
                self.v_for_stack.pop()
        else:
            widget = self._gen_widget(node)
            if self.parent_node_stack:
                self.parent_node_stack[-1]['body'].append(widget)
            else:
                self.widgets.append(widget)

    def compile(self, html):
        self.feed(html)
        if len(self.widgets) == 1:
            return self.widgets[0]
        return widgets.VBox(self.widgets)
        
# vue_template = VueTemplate(vue)
# app = vue_template.compile()
# display(app)

# Vue

In [16]:
class Options:
    def __init__(self, options):
        self.el = options.get('el')
        self.data = options.get('data')
        self.methods = options.get('methods', {})
        self.template = options.get('template')

        self.before_create = options.get('before_create')
        self.created = options.get('created')
        self.before_mount = options.get('before_mount')
        self.mounted = options.get('mounted')
        self.before_update = options.get('before_update ')
        self.updated = options.get('updated ')
        self.render = options.get('render')


class ReactiveField:
    def __init__(self, field_name, data):
        self.field_name = field_name
        self._data = data

    def __get__(self, instance, owner):
        return self._data[self.field_name]

    def __set__(self, instance: 'Vue', new_value):
        self._data[self.field_name] = new_value

        instance._call_if_callable(instance.options.before_update)
        instance.render()
        instance._call_if_callable(instance.options.updated)


class Dict(dict):
    pass

def _get_attr(scope, exp, default=None):
    attr_chain = exp.split('.')
    data = scope
    for attr in attr_chain:
        # TODO 可以用自定义的Nil替代，区分值为None的情况
        if isinstance(data, dict):
            data = data.get(attr, default)
        else:
            data = getattr(data, attr, default)
    return data

def get_attr_from_for_scope(scope: 'ForScope', exp: str, default=None):
    attr_split = exp.split('.', 1)
    if len(attr_split) == 1:
        attr_base, attrs = exp, ''
    else:
        attr_base, attrs = attr_split

    if attr_base == scope.for_stmt.iter:
        log_debug(f"get_attr_from_for iter {exp}  {attrs} from {scope.iter}")
        data_base = scope.iter
    elif attr_base == scope.for_stmt.target:
        log_debug(f"get_attr_from_for target {exp} {attrs} from {scope.target}")
        data_base = scope.target
        
    if attrs:
        data = _get_attr(data_base, attrs, default) 
    else:
        data = data_base
    return data

def get_attr(scope, exp, default=None):
    if isinstance(scope, ForScope):
        return get_attr_from_for_scope(scope, exp, default)
    return _get_attr(scope, exp, default)


def set_attr(scope, exp: str, value, reactive=False):
    attrs = exp.rsplit('.', 1)
    if len(attrs) == 1:
        attr_set = attrs[0]
        data = scope
    else:
        _attr_get, attr_set = attrs
        data = get_attr(scope, _attr_get)
        
    if data is None:
        return False

    if reactive:
        setattr(data, attr_set, value)
    else:
        if isinstance(data, Vue):
            data._data[attr_set] = value
        else:
            data[attr_set] = value

    return True

def get_attr_from_scopes(scopes, exp):
    for scope in scopes[::-1]:
        data = get_attr(scope, exp)
        if data is not None:
            return data

def set_attr_to_scopes(scopes, exp, value, reactive=False):
    for scope in scopes[::-1]:
        if set_attr(scope, exp, value, reactive):
            return True
    return False


class Vue:
    def __init__(self, options):
        self.dom = None
        self.options = Options(options)
        self._call_if_callable(self.options.before_create)
        self._data = self.options.data
        self._call_if_callable(self.options.created)
        self._proxy_data()
        self.methods: Methods = self.options.methods(self)
        self._proxy_methods()

        self.mount(self.options.el)

    def _call_if_callable(self, func):
        if callable(func):
            func(self)

    # TODO 全部代理
    def _proxy_data(self):
        for key, value in self._data.items():
            if isinstance(value, dict):
                item = Dict(value)
                setattr(self.__class__, key, ReactiveField(key, self._data))
                for sub_key, sub_value in value.items():
                    # sub_self = self._data[key]
                    # setattr(sub_self.__class__, sub_key, ReactiveField(sub_key, self._data[key]))
                    setattr(item.__class__, sub_key, ReactiveField(sub_key, item))
                self._data[key] = item
            elif isinstance(value, list) and isinstance(value[0], dict):
                for i, item in enumerate(value):
                    item = Dict(item)
                    for sub_key, sub_value in item.items():
                        setattr(item.__class__, sub_key, ReactiveField(sub_key, item))
                    value[i] = item
                # TODO 响应式的list
                setattr(self.__class__, key, value)
            else:
                log_debug(f'set {self.__class__.__name__} {key}')
                setattr(self.__class__, key, ReactiveField(key, self._data))
                log_debug(f'get {self.__class__.__name__} {key} {getattr(self, key)}')

    def _proxy_methods(self):
        pass
        # for func_name, func in self.options.methods.items():
        #     # TODO 绑定self
        #     # setattr(self, func_name, )
        #     pass

    def mount(self, el):
        self._call_if_callable(self.options.before_mount)
        self.render()
        self._call_if_callable(self.options.mounted)

    def _compile_template(self, template):
        vue_template = VueTemplate(self)
        return vue_template.compile(template)

    def render(self):
        # clear_output()
        with self.options.el:
            if callable(self.options.template):
                self.dom = self.options.template(self)
            else:
                self.dom = self._compile_template(self.options.template)
                
            clear_output(True)
            display(self.dom)

# template

In [1]:
html_hr = widgets.HTML('<hr>')
html_br = widgets.HTML('<br>')

class Role(enum.Enum):
    user = 'USER'
    bot = 'BOT'

    def __invert__(self):
        return Role.user if self == Role.bot else Role.bot


def update_value(func):
    def warp(change):
        func(change['new'])
    return warp


def update_list_value(index):
    def warp(func):
        def _(change):
            func(index, change['new'])
        return _
    return warp


def template(vm: Vue):
    # right sider bar
    # ==============================
    def _update_value(key):
        @update_value
        def update(value):
            vm._data[key] = value
        return update

    model = widgets.Dropdown(options=vm.models, value=vm._data['model'], tooltip='aaaa', description='model')
    model.observe(_update_value('model'), names='value')
        
    top_p = widgets.FloatSlider(value=vm._data['top_p'], min=0, max=1, step=0.01, description="top_p",
                                continuous_update=False, readout_format='.2f')
    top_p.observe(_update_value('top_p'), names='value')

    right_sidebar = widgets.VBox([
        model,
        top_p,
    ])
    # center
    # ==============================
    chat_exchange = []
    for i, msg in enumerate(vm._data['msgs']):
        @update_list_value(i)
        def update_role_label(index, value):
            vm._data['msgs'][index]['role'] = value

        def handle_role_label_on_click(owner):
            owner.description = (~Role(owner.description)).value

        msg_role = widgets.Button(description=msg['role'])
        msg_role.observe(update_role_label, names='description')
        msg_role.on_click(handle_role_label_on_click)

        def handle_del_msg_on_click(owner):
            vm.msgs.pop(owner.v_for_index)
            # TODO 实现自动更新
            vm.render()
        del_msg = widgets.Button(icon='minus-circle', button_style='danger')
        del_msg.v_for_index = i
        del_msg.on_click(handle_del_msg_on_click)

        @update_list_value(i)
        def update_msg_content(index, value):
            vm._data['msgs'][index]['content'] = value

        msg_content = widgets.Textarea(value=msg['content'])
        msg_content.observe(update_msg_content, names='value')

        msg_row = widgets.HBox([msg_role, msg_content, del_msg])
        chat_exchange.append(msg_row)
        chat_exchange.append(html_hr)

    def handle_add_msg_on_click(e):
        role = ~Role(vm.msgs[-1]['role']) if len(vm.msgs) else Role.user
        msg = {'role': role.value, 'content': ''}
        vm.msgs.append(msg)
        # TODO 实现自动更新
        vm.render()

    add_msg = widgets.Button(description='add msg', icon='plus-circle')
    add_msg.on_click(handle_add_msg_on_click)

    def handle_submit_msg_on_click(e):
        with vm.options.el:
            print('submit ', vm._data)

    submit_msg = widgets.Button(description='submit msg', button_style='success')
    submit_msg.on_click(handle_submit_msg_on_click)


    center = widgets.VBox([
        widgets.VBox(chat_exchange),
        add_msg,
        html_br,
        submit_msg,
    ])
    app = widgets.AppLayout(
        header=None,
        footer=None,
        left_sidebar=None,
        right_sidebar=right_sidebar,
        center=center
    )
    return app

NameError: name 'widgets' is not defined

In [18]:
# vue.msgs = vue.msgs + [{'role': 'USER', 'content': 'hello'}, {'role': 'USER', 'content': 'hello'}]

# 开发用示例 v2

# 开发用示例 v3

In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pathlib
import ipywidgets as widgets

from vuepy import create_app, import_sfc
from ipywui import IPywidgets

source_root = pathlib.Path().absolute().parent
App = import_sfc(source_root / 'examples' / 'vue_app'/ 'VueApp.vue')

app = create_app(App, debug=True).use(IPywidgets)
app.mount(widgets.Output())
# display(app.debug_log)

Output()

In [1]:
from pathlib import Path
import sys
import ipywidgets as widgets
from ipywui.widgets import ClipboardWidget

textarea = widgets.Textarea()
copy_btn = widgets.Button(description='copy')
clip = ClipboardWidget(tag='div', copy='hello', children=[textarea, copy_btn])
widgets.jslink((textarea, 'value'), (clip, 'copy'))
# clip.value = 'hello'
clip

/Users/kuroro/miniforge3/envs/textgen/lib/python3.10/site-packages/anywidget/_util.py:251: UserWarning: anywidget: Live-reloading feature is disabled. To enable, please install the 'watchfiles' package.
  start_thread=_should_start_thread(path),


Clipboard(children=[Textarea(value=''), Button(description='copy', style=ButtonStyle())], copy='hello')

In [2]:
# clip.value = '你好 hdsworrld '
a = 1
def handler(e):
    global a
    a = a+1
    print(e, a)
    
clip.observe(handler, names=['value'])

In [3]:
# handler(1)
print(a)

1


In [1]:
from pprint import pprint
import enum
from pyvue import *


class Role(enum.Enum):
    user = 'USER'
    bot = 'BOT'
    
    def __invert__(self):
        return Role.user if self == Role.bot else Role.bot


class Methods:
    def __init__(self, vm: 'Vue'):
        self.vm = vm

    def is_bot(self, val):
        return val == Role.bot.value

    def handle_change_role(self, owner):
        print(owner)
        owner.description = (~Role(owner.description)).value

    def handle_add_msg(self, owner):
        role = ~Role(self.vm.messages[-1]['role']) if len(self.vm.messages) else Role.user
        msg = {'role': role.value, 'content': ''}
        self.vm.messages.append(msg)
        # TODO 实现自动更新
        # self.vm.render()
    
    def handle_edit_msg(self, owner):
        if owner.icon == 'edit':
            state = 'eye'
        else:
            state = 'edit'
        owner.icon = state

    def handle_del_msg(self, i):
        with self.vm.options.el:
            msg = self.vm.messages.pop(i)
            # self.vm.render()
            print(f'del {i} {msg}')
            
    def handle_on_submit(self):
        with self.vm.options.el:
            clear_output()
            display(self.vm.dom)
            pprint(self.vm._data)


attr_chain_list = [
  ('op1', {
    'name': 'l1',
    'in': [1, 2, 4],
  }),
  ('op2', {
    'name': 'l2',
    'in': [5, 6, 7],
  }),
]

data = {
    'models': ['llama', 'llama2', 'llama2-chat'],
    'model': 'llama',
   
    'attr_options': attr_chain_list,
    'option': attr_chain_list[0][1],
 
    'top_p': 1,
    'messages': [
        {'role': Role.user.value, 'content': 'hello bot'},
        {'role': Role.bot.value, 'content': '## h2 \nhello user'},
        {'role': Role.user.value, 'content': 'world bot'},
    ],
    'on_edit': True,
    # 'msgs': [ {'role': Role.user.value, 'content': 'hello bot'}, ],
}
# with open('ipywidgets_vue/app.vue') as f:
#     vue_app = f.read()
#     str_template = re.search('<template>(.*)</template>', vue_app, flags=re.S)[1]
    
vue = Vue({
    'el': widgets.Output(),
    'data': data,
    'methods': Methods,
    'template': get_template_from_vue('ipywidgets_vue/app.vue'),
    # 'template': template,
}, debug=True)
display(vue.options.el)
display(vue.debug_log)

Output()

Output()

In [11]:
vue.messages[2]._deps

defaultdict(pyvue.Dep, {})

In [24]:
import time
for i in range(20):
    time.sleep(0.05)
    vue.messages[0]['content'] += 's'

In [17]:
vue.messages = [{'role': 'USER', 'content': 'replace bot'}]

In [35]:
vue.models = ['1', '2']

In [6]:
b1 = widgets.Button(description='b1', button_style='danger',layout=widgets.Layout(width='100%'))
b2 =  widgets.Button(description='b2', layout=widgets.Layout(width='100%'))
app = widgets.AppLayout(right_sidebar=b1, center=b2, pane_widths=[0,3,1.5])
app

AppLayout(children=(Button(button_style='danger', description='b1', layout=Layout(grid_area='right-sidebar', w…

In [8]:
app.children = [b1]

# 🚀 LLaMA2 dev GUI

In [1]:
import pathlib
import ipywidgets as widgets

from ipywui import IPywidgets
from vuepy import create_app, import_sfc

source_root = pathlib.Path().absolute().parent
App = import_sfc(source_root / 'examples' / 'llamacpp_dev' / 'LlamacppDev.vue')

app = create_app(App, debug=True).use(IPywidgets)
app.mount(widgets.Output())

Output()

# 🚀 Openai Playground Demo

In [3]:
# todo

In [1]:
class Dict(dict):
    def __getitem__(self, attr):
        print(f'get attr {attr}')
        return super().__getitem__(attr)
        
    def __getattr__(self, attr):
        return self.__getitem__(attr)
        
    def __setitem__(self, attr, value):
        print(f'set attr {attr}: {value}')
        super().__setitem__(attr, value)

    def __setattr__(self, attr, value):
        self.__setitem__(attr, value)

d = {'a': 1, 'b': {'c': 2}}
dd = Dict(d)
print('get[]')
dd['a']
print('set[]')
dd['a'] = 2
print('get')
dd.a
print('set')
dd.b['c'] = 1
print('print')
dd

get[]
get attr a
set[]
set attr a: 2
get
get attr a
set
get attr b
print
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr a
get attr b
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_
get attr _ipython_canary_method_should_not_exist_


{'a': 2, 'b': {'c': 1}}

将一个ipywidgets转换为vue component，

```python
ai> 将以下组件的参数转换为vue component的props定义：widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)

ai> 将https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html 中selection widgets的各个widgets的参数，转换为vue component的props定义，并标注出和这个widget最匹配的element-ui组件
```